In [41]:
import pandas as pd
import random
from itertools import product

# Load the data
moderator_data = pd.read_excel("./Scoring/moderator_scored.xlsx")

# Moderator Matching Algorithm
**Purpose:**
- Increase Revenue: By matching the best reviewers with the top priority ads, we ensure that high-quality ads get the attention they deserve, leading to better user experience and potentially higher revenue

- Improve Resource Utilization: By dynamically allocating tasks based on moderator capacity and expertise, we aim to increase the utilization of ad moderators by at most 10%

In [42]:
ads_dataset = pd.read_excel("./Scoring/ads_with_score.xlsx", nrows=10000)

# Gurobipy Optimization Model (Moderators)
1. **Decision Variables**
- For each ad and each moderator, a binary decision variable is created to represent whether the ad is assigned to the moderator (1) or not (0)

2. **Objective Function**
- The first objective function to minimize the difference between ad's score and moderator's score, this ensures that the top ads get matched with the best moderators
- The second objective function is with regards to the confidence of the ad, which is defined as how confident our model is in identifying whether there is a possible violation in the ad
    - If confidence is high, the ad is "easier" to moderator as it is easy to spot whether there is a violation. Hence the ad will be allocated to moderators that have high productivity so the ad can be cleared quickly, and low accuracy as a tradeoff
    - For ads with low confidence, it will be allocated to moderators that have high accuracy so that violations can be correctly identified, with the tradeoff being low productivity
- Combining these two objective functions, the overall objective function is as follows:
$$
\text{Minimize: } \sum_{\text{ad, mod}} \left| \beta_1 (ad\_score - moderator\_score) + \beta_2 (ad\_confidence - normalized\_productivity + normalized\_accuracy) \right|
$$

The coefficients $\beta_1$ and $\beta_2$ in the objective function represent weights that determine the importance of matching `ad_score` with `moderator_score` and `ad_confidence` with `normalized_productivity` and `normalized_accuracy`, respectively. Ideally, these coefficients should be derived from an iterative training and optimization process of the Gurobi model. By analyzing the results over multiple iterations and adjusting these coefficients, one can refine the model's performance and achieve better task allocations that meet specific business objectives.

However, due to constraints in our current setup, we are unable to conduct this iterative training. As a result, for the purpose of this demonstration, we are using placeholder values for $\beta_1$ and $\beta_2$, both assumed to be 0.5. This means that, in our current model, both parts of the objective function are given equal importance. 

3. **Constraints**
- One ad should only be allocated 1 moderator
- The total tasks allocated to each moderator cannot exceed their max_tasks_per_day
- The delivery_country of the ad must match the market of the moderator
- If moderator's category expertise is not null, expertise must match category of ad
- If null, each moderator can only be assigned a maximum of 3 ad categories a day
    - This is to ensure that the moderator's work is more focused to improve productivity

In [43]:
from gurobipy import Model, GRB

# Initialize the Gurobi model
m = Model("AdTaskAllocation")

# Create the decision variables
x = {(ad_id, mod): m.addVar(vtype=GRB.BINARY, name=f"x_{ad_id}_{mod}")
     for ad_id in ads_dataset['ad_id']
     for mod in moderator_data['moderator']}

In [44]:
# Set the objective function
m.setObjective(sum(x[ad_row['ad_id'], mod_row['moderator']] * abs(0.5 * (ad_row['ad_score'] - mod_row['moderator_score']) + 
    0.5 * (ad_row['confidence'] - mod_row['normalized_productivity'] + mod_row['normalized_accuracy'])) 
                for _, ad_row in ads_dataset.iterrows() for _, mod_row in moderator_data.iterrows()), GRB.MINIMIZE)

In [45]:
# Constraints 1-5

# Each ad should be allocated to only one moderator
for _, ad_row in ads_dataset.iterrows():
    ad_id = ad_row['ad_id']
    m.addConstr(sum(x[ad_id, mod_row['moderator']] for _, mod_row in moderator_data.iterrows()) == 1)

# The total tasks assigned to a moderator should not exceed their max tasks per day
for _, mod_row in moderator_data.iterrows():
    mod = mod_row['moderator']
    m.addConstr(sum(x[ad_row['ad_id'], mod] for _, ad_row in ads_dataset.iterrows()) <= mod_row['max_tasks_per_day'])

# Only assign an ad to a moderator if the ad's market matches the moderator's market

# Preprocess moderator data into a dictionary by market
moderator_market_dict = {}
for _, mod_row in moderator_data.iterrows():
    mod = mod_row['moderator']
    market = eval(mod_row['market'])
    for i in market:
        if i not in moderator_market_dict:
            moderator_market_dict[i] = []
        moderator_market_dict[i].append(mod)

# Check the moderator constraint more efficiently
for _, ad_row in ads_dataset.iterrows():
    ad_id = ad_row['ad_id']
    ad_market = ad_row['queue_market']
    
    # Get the list of moderators that match the ad's market from the preprocessed dictionary
    matching_mods = moderator_market_dict.get(ad_market, [])

    # Get all the moderators
    all_mods = moderator_data['moderator'].tolist()

    # For each moderator, if they are not in the matching moderators list, 
    # set their assignment variable for the current ad to 0.
    for mod in all_mods:
        if mod not in matching_mods:
            m.addConstr(x[ad_id, mod] == 0)

In [ ]:
# Solve the model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 309807 rows, 7710000 columns and 14441377 nonzeros
Model fingerprint: 0x3b741ddd
Variable types: 0 continuous, 7710000 integer (7710000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 4e+04]
Found heuristic solution: objective 4733.1660757
Presolve removed 302531 rows and 1608698 columns (presolve time = 7s) ...
Presolve removed 303528 rows and 1610853 columns (presolve time = 11s) ...
Presolve removed 303528 rows and 1610853 columns (presolve time = 15s) ...
Presolve removed 303528 rows and 1610853 columns (presolve time = 21s) ...
Presolve removed 303528 rows and 1610853 columns (presolve time = 25s) ...
Presolve removed 303528 rows and 1610853 columns (presolve time = 31s) ...
Presolve re

In [ ]:
# Extract the assignments from the solution
assignments = {}
for _, ad_row in ads_dataset.iterrows():
    for _, mod_row in moderator_data.iterrows():
        if x[ad_row['ad_id'], mod_row['moderator']].x > 0.5:  # If this ad is assigned to this moderator
            assignments[ad_row['ad_id']] = mod_row['moderator']

assignments

{1747578422390810: 1702615776441346,
 1758543228094480: 1752090693569553,
 1738303151826990: 3371590,
 1738303920938010: 2613990,
 1738304016410620: 7767295,
 1738304421208090: 7767295,
 1738304605976620: 7767295,
 1724385503656970: 5858713,
 1764813273437210: 5858713,
 1763583923607580: 5858713,
 1764585402541070: 1767847119851538,
 1765089254920220: 1767847119851538,
 1765121170665500: 1767847119851538,
 1763582835715070: 5338213,
 1763766858169370: 5338213,
 1767664297198600: 1767847119851538,
 1767664297196540: 1767847119851538,
 1767540892154930: 1767847119851538,
 1767027023555630: 1736487777311745,
 1767187528079370: 7767295,
 1767187070801970: 7767295,
 1767187070787580: 7767295,
 1767187070795790: 7767295,
 1767392366439450: 5858713,
 1760747822381050: 5858713,
 1760794835125260: 1398796,
 1767567705104400: 1767847119851538,
 1760966428402700: 6085577,
 1768110086084620: 5338213,
 1768110086089740: 5338213,
 1763922466157560: 9152229,
 1763922466156570: 7615952,
 1769057503455

In [ ]:
# Convert the dictionary of assignments into a DataFrame
assignments_df = pd.DataFrame(list(assignments.items()), columns=['ad_id', 'moderator'])

# Count the number of ads assigned to each moderator
assigned_counts = assignments_df.groupby('moderator').size().reset_index(name='num_ads_assigned')

# Merge with the original moderator_data
merged_data = pd.merge(moderator_data, assigned_counts, left_on='moderator', right_on='moderator', how='left')

# Fill NaN values with 0 (for moderators with no assignments)
merged_data['num_ads_assigned'].fillna(0, inplace=True)

# Calculate the increase in utilization % and new utilization % for each moderator
merged_data['increase_in_utilisation'] = (merged_data['num_ads_assigned'] * merged_data['handling time']) / (8 * 60 * 60 * 1000)
merged_data['new_utilisation'] = merged_data['Utilisation %'] + merged_data['increase_in_utilisation']

# Extract the relevant columns for display
output_table = merged_data[['moderator', 'num_ads_assigned', 'increase_in_utilisation', 'new_utilisation', 'max_tasks_per_day']].sort_values(['new_utilisation'])

output_table = output_table[output_table['num_ads_assigned'] > 0]

output_table

,moderator,num_ads_assigned,increase_in_utilisation,new_utilisation,max_tasks_per_day
1269,1755154458300433,26.0,0.094089,0.172417,27.633321
1260,1295012,30.0,0.099158,0.257539,30.254643
1228,6587921,30.0,0.083836,0.345842,35.783954
1229,2097663,81.0,0.099332,0.358118,81.544821
1194,1912879,3.0,0.012012,0.359699,24.975718
...,...,...,...,...,...
15,5569930,25.0,0.099157,1.145313,25.212512
9,2322011,20.0,0.064888,1.150040,30.822578
7,7378621,12.0,0.039560,1.152442,30.333670
11,1752090693569553,25.0,0.097431,1.165419,25.659073
